In [271]:
from typing import List

# Initial Steps

### Create plaintext (first student's full name)

In [275]:
plaintext_name = "Saman Gilani"
plaintext_16 = plaintext_name.ljust(16)  # pad to 16 characters
plaintext_hex = [hex(ord(c)) for c in plaintext_16]
print("Plaintext (16 chars):", repr(plaintext_16))
print("Plaintext ASCII → Hex:", plaintext_hex)

Plaintext (16 chars): 'Saman Gilani    '
Plaintext ASCII → Hex: ['0x53', '0x61', '0x6d', '0x61', '0x6e', '0x20', '0x47', '0x69', '0x6c', '0x61', '0x6e', '0x69', '0x20', '0x20', '0x20', '0x20']


### Create master key (second student's full name)

In [278]:
key_name = "Saira Ali Bhatti"
key_16 = key_name[:16]  # already 16 characters
key_hex = [hex(ord(c)) for c in key_16]
print("\nMaster Key (16 chars):", repr(key_16))
print("Master Key ASCII → Hex:", key_hex)


Master Key (16 chars): 'Saira Ali Bhatti'
Master Key ASCII → Hex: ['0x53', '0x61', '0x69', '0x72', '0x61', '0x20', '0x41', '0x6c', '0x69', '0x20', '0x42', '0x68', '0x61', '0x74', '0x74', '0x69']


In [280]:
plaintext_hex = "".join(f"{ord(c):02x}" for c in plaintext_16)
print("Plaintext (16 chars):", repr(plaintext_16))
print("Plaintext Hex:", plaintext_hex)

key_hex = "".join(f"{ord(c):02x}" for c in key_16)
print("\nMaster Key (16 chars):", repr(key_16))
print("Master Key Hex:", key_hex)

Plaintext (16 chars): 'Saman Gilani    '
Plaintext Hex: 53616d616e2047696c616e6920202020

Master Key (16 chars): 'Saira Ali Bhatti'
Master Key Hex: 536169726120416c6920426861747469


### Helpers, S-box, Key Expansion, AES primitives

In [283]:
SBOX = [
0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]

In [285]:
RCON = [0x00,0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]

In [287]:
def str_to_16bytes(s):
    s = s.ljust(16)[:16]
    return [ord(c) for c in s]

def bytes_to_hex(b):
    return "".join(f"{x:02x}" for x in b)

def bytes_to_state(b):
    return [[b[c*4+r] for r in range(4)] for c in range(4)]

def state_to_bytes(state):
    out = []
    for c in range(4):
        for r in range(4):
            out.append(state[c][r])
    return out

def add_round_key(state, rk_bytes):
    rk_state = bytes_to_state(rk_bytes)
    return [[state[c][r]^rk_state[c][r] for r in range(4)] for c in range(4)]

def sub_bytes(state):
    return [[SBOX[state[c][r]] for r in range(4)] for c in range(4)]

def shift_rows(state):
    rows = [[state[c][r] for c in range(4)] for r in range(4)]
    for r in range(4):
        rows[r] = rows[r][r:] + rows[r][:r]
    return [[rows[r][c] for r in range(4)] for c in range(4)]

def xtime(x):
    return ((x << 1) & 0xff) ^ (0x1b if (x & 0x80) else 0x00)

def mix_single_column(col):
    a = col[:]
    b = [xtime(x) for x in a]
    return [
        b[0]^a[3]^a[2]^b[1]^a[1],
        b[1]^a[0]^a[3]^b[2]^a[2],
        b[2]^a[1]^a[0]^b[3]^a[3],
        b[3]^a[2]^a[1]^b[0]^a[0],
    ]

def mix_columns(state):
    return [mix_single_column(state[c]) for c in range(4)]

def rot_word(w):
    return w[1:] + w[:1]

def sub_word(w):
    return [SBOX[b] for b in w]

def key_expansion(key_bytes):
    Nk = 4
    Nr = 10
    w = [key_bytes[i*4:(i+1)*4] for i in range(Nk)]
    for i in range(Nk, 4*(Nr+1)):
        temp = w[i-1][:]
        if i % Nk == 0:
            temp = sub_word(rot_word(temp))
            temp[0] ^= RCON[i//Nk]
        w.append([ (w[i-Nk][j] ^ temp[j]) & 0xff for j in range(4)])
    round_keys = []
    for r in range(Nr+1):
        rk = []
        for j in range(4):
            rk += w[r*4+j]
        round_keys.append(rk)
    return round_keys

## Task (a): Derive Round Keys K0 and K1

In [290]:
def task_a(master_key_str):
    key_bytes = str_to_16bytes(master_key_str)
    round_keys = key_expansion(key_bytes)
    
    print("Initial Key K0:")
    print(bytes_to_hex(round_keys[0]))
    print("\nRound Key K1:")
    print(bytes_to_hex(round_keys[1]))
    
    print("\nAll Round Keys K0..K10:")
    for i, rk in enumerate(round_keys):
        print(f"K{i:2d}: {bytes_to_hex(rk)}")

task_a("Saira Ali Bhatti")

Initial Key K0:
536169726120416c6920426861747469

Round Key K1:
c0f3909da1d3d1f1c8f39399a987e7f0

All Round Keys K0..K10:
K 0: 536169726120416c6920426861747469
K 1: c0f3909da1d3d1f1c8f39399a987e7f0
K 2: d5671c4e74b4cdbfbc475e2615c0b9d6
K 3: 6b31ea171f8527a8a3c2798eb602c058
K 4: 148b80590b0ea7f1a8ccde7f1ece1e27
K 5: 8ff94c2b84f7ebda2c3b35a532f52b82
K 6: 49085f08cdffb4d2e1c48177d331aaf5
K 7: cea4b96e035b0dbce29f8ccb31ae263e
K 8: aa530ba9a90806154b978ade7a39ace0
K 9: a3c2ea730acaec66415d66b83b64ca58
K10: d6b68091dc7c6cf79d210a4fa645c017


## Task (b): Show the initial AddRoundKey.

In [265]:
plaintext_bytes = [ord(c) for c in plaintext_16]
key_bytes = [ord(c) for c in key_16]
k0 = key_bytes

# Initial AddRoundKey = plaintext XOR K0
initial_state = [p ^ k for p, k in zip(plaintext_bytes, K0)]

# Function to print AES 4x4 matrix
def print_matrix(state, title):
    print(title)
    for r in range(4):
        row = state[r*4:(r+1)*4]
        print(" ".join(f"{x:02x}" for x in row))
    print()

print_matrix(initial_state, "Initial AddRoundKey (K0):")


Initial AddRoundKey (K0):
00 00 04 13
0f 00 06 05
05 41 2c 01
41 54 54 49



### Task (c): Perform Round 1 (SubBytes, ShiftRows, MixColumns, AddRoundKey with K1).

In [267]:
def show_round1(plaintext_str, master_key_str):
    pt_bytes = str_to_16bytes(plaintext_str)
    key_bytes = str_to_16bytes(master_key_str)
    round_keys = key_expansion(key_bytes)
    
    state_ark0 = add_round_key(bytes_to_state(pt_bytes), round_keys[0])
    state_sub = sub_bytes(state_ark0)
    state_sr = shift_rows(state_sub)
    state_mc = mix_columns(state_sr)
    state_ark1 = add_round_key(state_mc, round_keys[1])
    
    def print_table(title, s):
        print(title)
        rows = [[s[c][r] for c in range(4)] for r in range(4)]
        for r in range(4):
            print(" ".join(f"{x:02x}" for x in rows[r]))
        print()
    
    print_table("Initial AddRoundKey (K0):", state_ark0)
    print_table("After SubBytes:", state_sub)
    print_table("After ShiftRows:", state_sr)
    print_table("After MixColumns:", state_mc)
    print_table("After AddRoundKey (K1):", state_ark1)

# ---- Run Round 1 ----
show_round1("Saman Gilani", "Saira Ali Bhatti")

Initial AddRoundKey (K0):
00 0f 05 41
00 00 41 54
04 06 2c 54
13 05 01 49

After SubBytes:
63 76 6b 83
63 63 83 20
f2 6f 71 20
7d 6b 7c 3b

After ShiftRows:
63 76 6b 83
63 83 20 63
71 20 f2 6f
3b 7d 6b 7c

After MixColumns:
29 2f 2f ab
0d 76 4d 88
af 32 09 ba
c1 c3 b9 6a

After AddRoundKey (K1):
e9 8e e7 02
fe a5 be 0f
3f e3 9a 5d
5c 32 20 9a



## Task (d): Full AES encryption, final ciphertext

In [269]:
def aes_encrypt_full(pt_str, master_key_str):
    pt_bytes = str_to_16bytes(pt_str)
    key_bytes = str_to_16bytes(master_key_str)
    round_keys = key_expansion(key_bytes)
    
    state = add_round_key(bytes_to_state(pt_bytes), round_keys[0])
    
    for r in range(1, 10):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, round_keys[r])
    
    # Final round 10 (no MixColumns)
    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, round_keys[10])
    
    ciphertext = state_to_bytes(state)
    print("Final Ciphertext (hex):", bytes_to_hex(ciphertext))

# ---- Run Full AES ----
aes_encrypt_full("Saman Gilani", "Saira Ali Bhatti")


Final Ciphertext (hex): f6f3820493cd32c409267481ecb0d7dd


### Verify with python library

In [296]:
pip install pycryptodome

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 1.9 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [302]:
from Crypto.Cipher import AES

# Plaintext and key (16 bytes each)
plaintext = b'Saman Gilani    '  # 16 characters
key = b'Saira Ali Bhatti'        # 16 characters

# Create AES cipher in ECB mode
cipher = AES.new(key, AES.MODE_ECB)

# Encrypt
ciphertext = cipher.encrypt(plaintext)

# Print ciphertext in hex
print("Ciphertext (hex):", ciphertext.hex())

Ciphertext (hex): f6f3820493cd32c409267481ecb0d7dd


In [304]:
from Crypto.Cipher import AES

# Ciphertext in hex (from your encryption)
ciphertext_hex = 'f6f3820493cd32c409267481ecb0d7dd'
ciphertext = bytes.fromhex(ciphertext_hex)

# Same key used for encryption
key = b'Saira Ali Bhatti'  # 16 bytes

# Create AES cipher in ECB mode
cipher = AES.new(key, AES.MODE_ECB)

# Decrypt
decrypted = cipher.decrypt(ciphertext)

# Print original plaintext
print("Decrypted text:", decrypted.decode())


Decrypted text: Saman Gilani    


## AES Decryption Process

In [306]:
# Inverse S-box (AES standard)
InvSBox = [
    0x52,0x09,0x6a,0xd5,0x30,0x36,0xa5,0x38,0xbf,0x40,0xa3,0x9e,0x81,0xf3,0xd7,0xfb,
    0x7c,0xe3,0x39,0x82,0x9b,0x2f,0xff,0x87,0x34,0x8e,0x43,0x44,0xc4,0xde,0xe9,0xcb,
    0x54,0x7b,0x94,0x32,0xa6,0xc2,0x23,0x3d,0xee,0x4c,0x95,0x0b,0x42,0xfa,0xc3,0x4e,
    0x08,0x2e,0xa1,0x66,0x28,0xd9,0x24,0xb2,0x76,0x5b,0xa2,0x49,0x6d,0x8b,0xd1,0x25,
    0x72,0xf8,0xf6,0x64,0x86,0x68,0x98,0x16,0xd4,0xa4,0x5c,0xcc,0x5d,0x65,0xb6,0x92,
    0x6c,0x70,0x48,0x50,0xfd,0xed,0xb9,0xda,0x5e,0x15,0x46,0x57,0xa7,0x8d,0x9d,0x84,
    0x90,0xd8,0xab,0x00,0x8c,0xbc,0xd3,0x0a,0xf7,0xe4,0x58,0x05,0xb8,0xb3,0x45,0x06,
    0xd0,0x2c,0x1e,0x8f,0xca,0x3f,0x0f,0x02,0xc1,0xaf,0xbd,0x03,0x01,0x13,0x8a,0x6b,
    0x3a,0x91,0x11,0x41,0x4f,0x67,0xdc,0xea,0x97,0xf2,0xcf,0xce,0xf0,0xb4,0xe6,0x73,
    0x96,0xac,0x74,0x22,0xe7,0xad,0x35,0x85,0xe2,0xf9,0x37,0xe8,0x1c,0x75,0xdf,0x6e,
    0x47,0xf1,0x1a,0x71,0x1d,0x29,0xc5,0x89,0x6f,0xb7,0x62,0x0e,0xaa,0x18,0xbe,0x1b,
    0xfc,0x56,0x3e,0x4b,0xc6,0xd2,0x79,0x20,0x9a,0xdb,0xc0,0xfe,0x78,0xcd,0x5a,0xf4,
    0x1f,0xdd,0xa8,0x33,0x88,0x07,0xc7,0x31,0xb1,0x12,0x10,0x59,0x27,0x80,0xec,0x5f,
    0x60,0x51,0x7f,0xa9,0x19,0xb5,0x4a,0x0d,0x2d,0xe5,0x7a,0x9f,0x93,0xc9,0x9c,0xef,
    0xa0,0xe0,0x3b,0x4d,0xae,0x2a,0xf5,0xb0,0xc8,0xeb,0xbb,0x3c,0x83,0x53,0x99,0x61,
    0x17,0x2b,0x04,0x7e,0xba,0x77,0xd6,0x26,0xe1,0x69,0x14,0x63,0x55,0x21,0x0c,0x7d
]

# XOR two states
def AddRoundKey(state, round_key):
    return [[state[r][c] ^ round_key[r][c] for c in range(4)] for r in range(4)]

# Inverse ShiftRows
def InvShiftRows(state):
    state[1] = state[1][-1:] + state[1][:-1]
    state[2] = state[2][-2:] + state[2][:-2]
    state[3] = state[3][-3:] + state[3][:-3]
    return state

# Inverse SubBytes
def InvSubBytes(state):
    return [[InvSBox[byte] for byte in row] for row in state]

# Helper for InvMixColumns
def gmul(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        hi_bit = a & 0x80
        a <<= 1
        a &= 0xFF
        if hi_bit:
            a ^= 0x1b
        b >>= 1
    return p

# Inverse MixColumns
def InvMixColumns(state):
    for c in range(4):
        a = [state[r][c] for r in range(4)]
        state[0][c] = gmul(a[0],0x0e) ^ gmul(a[1],0x0b) ^ gmul(a[2],0x0d) ^ gmul(a[3],0x09)
        state[1][c] = gmul(a[0],0x09) ^ gmul(a[1],0x0e) ^ gmul(a[2],0x0b) ^ gmul(a[3],0x0d)
        state[2][c] = gmul(a[0],0x0d) ^ gmul(a[1],0x09) ^ gmul(a[2],0x0e) ^ gmul(a[3],0x0b)
        state[3][c] = gmul(a[0],0x0b) ^ gmul(a[1],0x0d) ^ gmul(a[2],0x09) ^ gmul(a[3],0x0e)
    return state

# Convert 16-byte hex into 4x4 state matrix
def hex_to_state(hex_str):
    bytes_list = [int(hex_str[i:i+2],16) for i in range(0,len(hex_str),2)]
    return [[bytes_list[r + 4*c] for c in range(4)] for r in range(4)]

# Convert state matrix back to plaintext string
def state_to_text(state):
    flat = [state[r][c] for c in range(4) for r in range(4)]
    return ''.join(chr(b) for b in flat)

# Example: your ciphertext and round keys
ciphertext_hex = 'f6f3820493cd32c409267481ecb0d7dd'
state = hex_to_state(ciphertext_hex)

# Round keys (K0..K10) you provided (fill as 4x4 matrices)
round_keys_hex = [
    "536169726120416c6920426861747469",
    "c0f3909da1d3d1f1c8f39399a987e7f0",
    "d5671c4e74b4cdbfbc475e2615c0b9d6",
    "6b31ea171f8527a8a3c2798eb602c058",
    "148b80590b0ea7f1a8ccde7f1ece1e27",
    "8ff94c2b84f7ebda2c3b35a532f52b82",
    "49085f08cdffb4d2e1c48177d331aaf5",
    "cea4b96e035b0dbce29f8ccb31ae263e",
    "aa530ba9a90806154b978ade7a39ace0",
    "a3c2ea730acaec66415d66b83b64ca58",
    "d6b68091dc7c6cf79d210a4fa645c017"
]

# Convert round keys to states
round_keys = [hex_to_state(k) for k in round_keys_hex]

# Initial AddRoundKey with last key
state = AddRoundKey(state, round_keys[10])

# 9 main rounds (InvShiftRows, InvSubBytes, AddRoundKey, InvMixColumns)
for round in range(9,0,-1):
    state = InvShiftRows(state)
    state = InvSubBytes(state)
    state = AddRoundKey(state, round_keys[round])
    state = InvMixColumns(state)

# Final round (no InvMixColumns)
state = InvShiftRows(state)
state = InvSubBytes(state)
state = AddRoundKey(state, round_keys[0])

# Print plaintext
plaintext = state_to_text(state)
print("Decrypted plaintext:", plaintext)


Decrypted plaintext: Saman Gilani    
